# What is an Approximate Functional Dependency?

In Desbordante we consider an approximate functional dependency ($AFD$)
any kind of functional dependency ($FD$) that employs an error metric and is not named (e.g. *soft functional dependencies*).

This metric is used to calculate the extent of violation for a given exact $FD$ and lies within `[0, 1]` range (the lower, the less violations
are found in data).

For the discovery task a user can specify the threshold and Desbordante
will find all $AFDs$, which have their error equal or less than the threshold, according to the selected metric.

# What we have to offer

Currently, Desbordante supports:
1. Five metrics: `g1`, `pdep`, `tau`, `mu+`, `rho`.
2. Two algorithms for discovery of $AFDs$: `Tane` and `Pyro`, with `Pyro` being the fastest.

  *Unfortunately, Pyro can handle only the g1 metric, for the rest use Tane.*

For more information consider:
1. *Measuring Approximate Functional Dependencies: A Comparative Study by M. Parciak et al.*
2. *Efficient Discovery of Approximate Dependencies by S. Kruse and F. Naumann.*
3. *TANE: An Efficient Algorithm for Discovering Functional and Approximate Dependencies by Y. Huhtala et al.*

## Mining example

Now, we are going to demonstrate how to discover $AFDs$.

First, install dependencies, import the modules and load the dataset.

In [1]:
!pip install desbordante==2.3.2
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 19.9 MB/s eta 0:00:00


In [2]:
import desbordante as db
import pandas as pd

In [3]:
!wget -q https://raw.githubusercontent.com/Desbordante/desbordante-core/main/examples/datasets/inventory_afd.csv
!wget -q https://raw.githubusercontent.com/Desbordante/desbordante-core/main/examples/datasets/DnD.csv

Display the dataset using `pandas`.

In [4]:
df = pd.read_csv("inventory_afd.csv")
df

,Id,ProductName,Price
0,1,Laptop,3000
1,2,Laptop,3000
2,3,Laptop,300
3,4,Laptop,3000
4,5,Smartwatch,600
5,6,Headphones,500
6,7,Tablet,300
7,8,Tablet,500
8,9,Smartphone,1000
9,10,Headphones,500


---
AFDs mined by Pyro

In [5]:
pyro_alg = db.afd.algorithms.Default()
pyro_alg.load_data(table=df)
pyro_alg.execute(error=0.3)

for fd in pyro_alg.get_fds():
    print(fd)

[ProductName] -> Id
[Price] -> Id
[Id] -> ProductName
[Price] -> ProductName
[Id] -> Price
[ProductName] -> Price


---
AFDs mined by Tane

In [6]:
ERROR_MEASURES = ['g1','pdep','tau','mu_plus', 'rho']

tane_alg = db.afd.algorithms.Tane()
tane_alg.load_data(table=df)

for MEASURE in ERROR_MEASURES:
    tane_alg.execute(error=0.3, afd_error_measure=MEASURE)
    result = tane_alg.get_fds()
    print(MEASURE+':')
    for fd in result:
        print(fd)
    print()

g1:
[Id] -> ProductName
[Id] -> Price
[ProductName] -> Id
[Price] -> Id
[ProductName] -> Price
[Price] -> ProductName

pdep:
[Id] -> ProductName
[Id] -> Price
[ProductName] -> Price

tau:
[Id] -> ProductName
[Id] -> Price

mu_plus:
[Id] -> ProductName
[Id] -> Price

rho:
[Id] -> ProductName
[Id] -> Price
[ProductName] -> Price



## Verification example

Now let's look at the `DnD.csv`.

In [7]:
data = pd.read_csv("DnD.csv", header=[0])
data

,Creature,Strength,HaveMagic
0,Ogre,9,False
1,Ogre,6,False
2,Elf,6,True
3,Elf,6,True
4,Elf,1,True
5,Dwarf,9,False
6,Dwarf,6,False


In [10]:
def print_clusters(verifier, data, lhs, rhs):
    print(f"Number of clusters violating FD: {verifier.get_num_error_clusters()}")
    for i, highlight in enumerate(verifier.get_highlights(), start=1):
        print(f"#{i} cluster: ")
        for el in highlight.cluster:
            print(f"\t{el}: {data[data.columns[lhs]][el]} -> {data[data.columns[rhs]][el]}")

        print(f"Most frequent rhs value proportion: {highlight.most_frequent_rhs_value_proportion}")
        print(f"Num distinct rhs values: {highlight.num_distinct_rhs_values}\n")

def print_results_for_fd(verifier, data, lhs, rhs):
    if verifier.fd_holds():
        print("FD holds")
    else:
        print("FD does not hold")
        print_clusters(verifier, data, lhs, rhs)

algo = db.afd_verification.algorithms.Default()
algo.load_data(table=data)
algo.execute(lhs_indices=[0], rhs_indices=[1])

In [12]:
def print_results_for_afd(verifier, error):
    if verifier.get_error() < error:
        print("AFD with this error threshold holds")
    else:
        print("AFD with this error threshold does not hold")
        print(f"But the same AFD with error threshold = {verifier.get_error()} holds")

Checking whether `[Creature]` $\rightarrow$ `[Strength]` AFD holds (error threshold = 0.5).

In [13]:
print_results_for_afd(algo, 0.5)

AFD with this error threshold holds


Checking whether `[Creature]` $\rightarrow$ `[Strength]` AFD holds (error threshold = 0.1)


In [14]:
print_results_for_afd(algo, 0.1)

AFD with this error threshold does not hold
But the same AFD with error threshold = 0.19047619047619047 holds


Similarly to the FD verification primitive, the AFD one can provide a user with clusters.

In [15]:
print_clusters(algo, data, 0, 1)

Number of clusters violating FD: 3
#1 cluster: 
	2: Elf -> 6
	3: Elf -> 6
	4: Elf -> 1
Most frequent rhs value proportion: 0.6666666666666666
Num distinct rhs values: 2

#2 cluster: 
	0: Ogre -> 9
	1: Ogre -> 6
Most frequent rhs value proportion: 0.5
Num distinct rhs values: 2

#3 cluster: 
	5: Dwarf -> 9
	6: Dwarf -> 6
Most frequent rhs value proportion: 0.5
Num distinct rhs values: 2

